In [1]:
import pandas as pd, numpy as np
np.set_printoptions(suppress=True)# 关掉科学计数法
import glob
import os
import csv
# 一次性merge多个pct_chg
from functools import reduce
from datetime import datetime, timedelta
import statsmodels.api as sm
from statsmodels import regression
import time, urllib

# import matplotlib
import matplotlib.pyplot as plt

import seaborn as sns
%matplotlib inline
sns.set(context='notebook', style='darkgrid', palette='deep', font='sans-serif', font_scale=1, color_codes=False, rc=None)

# 1. MACD系列

In [2]:
# 定义每个筛选模型

# 1. MACD
def MACD(df, price_col, n_fast, n_slow): # n_fast = 12, n_slow = 26
    """
    http://stockcharts.com/docs/doku.php?id=scans:indicators
    MACD, MACD Signal and MACD difference, rationale CHECKED, code CHECKED, updated
    # Conventional look-back window for calculating MACDsign is 9
    """
    EMAfast = df[price_col].ewm(span = n_fast, min_periods = n_fast - 1).mean()
    EMAslow = df[price_col].ewm(span = n_slow, min_periods = n_slow - 1).mean()
    diff = pd.Series(EMAfast - EMAslow)
    dea = diff.ewm(span = 9, min_periods = 8).mean()
    macd = (pd.Series(diff - dea))*2
    df["DIFF"] = diff
    df["DEA"] = dea
    df["MACD"] = macd
    return df

In [3]:
# 在MACD模型下的开平仓算法

def macd_updown_signals(df, signal_spread_col, date_col, ticker_col):
    ticker = df[ticker_col].values[-1]
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    del df['index']
    listLongShort = []
    listDate = []
    macd = df[signal_spread_col].values
    
    for i in range(1, len(df)):
        last_date = df[date_col][i]
        
        if macd[i]>macd[i-1] and macd[i-1]<macd[i-2]:
            
            listLongShort.append("BUY")
            listDate.append(last_date)
        #                          # The other way around
        elif macd[i]<macd[i-1] and macd[i-1]>macd[i-2]:
            listLongShort.append("SELL")
            listDate.append(last_date)
        #                          # Do nothing if not satisfied
        elif macd[i]<macd[i-1]:
            listLongShort.append("HOLD SHORT")
            listDate.append(last_date)
            
        elif macd[i]>macd[i-1]:
            listLongShort.append("HOLD LONG")
            listDate.append(last_date)           
            
    return ticker, listDate[-1], listLongShort[-1]
#     df['Advice'] = listLongShort
    # The advice column means "Buy/Sell/Hold" at the end of this day or
    #  at the beginning of the next day, since the market will be closed

In [4]:
def macd_cross_signals(df, signal_spread_col, date_col, ticker_col):
    ticker = df[ticker_col].values[-1]
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    del df['index']
    listLongShort = []
    listDate = []
    macd = df[signal_spread_col].values
    
    for i in range(1, len(df)):
        last_date = df[date_col][i]
        #                          # If the MACD crosses the signal line upward
        if macd[i] >0 and macd[i - 1] <0:
            listLongShort.append("BUY")
            listDate.append(last_date)
        #                          # The other way around
        elif macd[i] < 0 and macd[i - 1] >0:
            listLongShort.append("SELL")
            listDate.append(last_date)
        #                          # Do nothing if not crossed
        else: # 还要改，再增加点条件
            listLongShort.append("HOLD")
            listDate.append(last_date)
#     print("Ticker: ", ticker)
#     print("Last Date", listDate[-1])
#     print("Last Signal", listLongShort[-1])
    return ticker, listDate[-1], listLongShort[-1]
#     df['Advice'] = listLongShort
    # The advice column means "Buy/Sell/Hold" at the end of this day or
    #  at the beginning of the next day, since the market will be closed

## 日线 - MACD

In [5]:
all_csvs = "C:/Users/Administrator/CE_github_2/data_pipeline/Data/*.csv"
stock_list = []
date_list = []
signal_list = []
len_ = 1
date_col = "Date"
price_col = "Adj Close"
model_freq = "D"
signal_df = pd.DataFrame()
model_name = "Macd_Updown_Signals"
total_number = len(glob.glob(all_csvs)[:])


for fname in glob.glob(all_csvs)[:]:
    print ("Calculating Signals for No.{} : {}".format(len_, fname))
    data = pd.read_csv(fname)
    try:
        data_macd = MACD(data, "Adj Close", 12, 26)
        ticker, last_date, last_signal = macd_updown_signals(data_macd, "MACD", "Date", 'Ticker')
        stock_list.append(ticker)
        date_list.append(last_date)
        signal_list.append(last_signal)
    except Exception as e:
        print(e)
    len_+= 1
signal_df['Ticker'] = stock_list
signal_df['Last_Date'] = date_list
signal_df['Signal'] = signal_list
signal_df['model_name'] = model_name
signal_df['model_freq'] = model_freq
signal_df_daily = signal_df.copy()
print("All Done!")
# def merge_df(df1, df2):
#     df1.sort_values(date_col, inplace = True)
#     merged = df1.merge(df2, on = date_col, how = 'outer')
#     merged.sort_values(date_col, inplace = True)
#     return merged

# merged_all = reduce(merge_df, stock_list)
# merged_all.set_index(date_col, inplace=True)

Calculating Signals for No.1 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\000001.SS.csv
Calculating Signals for No.2 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0005.HK.csv
Calculating Signals for No.3 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0008.HK.csv
Calculating Signals for No.4 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0142.HK.csv
Calculating Signals for No.5 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0151.HK.csv
Calculating Signals for No.6 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0175.HK.csv
Calculating Signals for No.7 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0186.HK.csv
Calculating Signals for No.8 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0215.HK.csv
Calculating Signals for No.9 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0241.HK.csv
Calculating Signals for No.10 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0268.HK.csv
Calculating Signals for No.

Calculating Signals for No.92 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\ADSK.csv
Calculating Signals for No.93 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\AEMD.csv
Calculating Signals for No.94 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\AHPI.csv
Calculating Signals for No.95 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\ALB.csv
Calculating Signals for No.96 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\AMD.csv
Calculating Signals for No.97 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\AMZN.csv
Calculating Signals for No.98 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\APT.csv
Calculating Signals for No.99 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\ARKF.csv
Calculating Signals for No.100 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\ARKG.csv
Calculating Signals for No.101 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\ARKK.csv
Calculating Signals for No.102 : C:/Users/Administr

Calculating Signals for No.180 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\QQQ.csv
Calculating Signals for No.181 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\QS.csv
Calculating Signals for No.182 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\RENN.csv
Calculating Signals for No.183 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\RIOT.csv
Calculating Signals for No.184 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\RNG.csv
Calculating Signals for No.185 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\RUN.csv
Calculating Signals for No.186 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SAP.csv
Calculating Signals for No.187 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SBE.csv
Calculating Signals for No.188 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SHOP.csv
Calculating Signals for No.189 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SMAR.csv
Calculating Signals for No.190 : C:/Users/Admin

In [7]:
signal_path = "C:/Users/Administrator/CE_github_2/data_pipeline/Signals/"
signal_file = signal_path+model_name+"_"+last_date+".csv"
signal_df_daily.to_csv(signal_file)

In [8]:
signal_df_daily

,Ticker,Last_Date,Signal,model_name,model_freq
0,000001.SS,2021-02-26,HOLD SHORT,Macd_Updown_Signals,D
1,0005.HK,2021-02-26,SELL,Macd_Updown_Signals,D
2,0008.HK,2021-02-26,HOLD SHORT,Macd_Updown_Signals,D
3,0142.HK,2021-02-26,SELL,Macd_Updown_Signals,D
4,0151.HK,2021-02-26,HOLD SHORT,Macd_Updown_Signals,D
...,...,...,...,...,...
223,ZEN,2021-02-26,HOLD SHORT,Macd_Updown_Signals,D
224,ZM,2021-02-26,HOLD SHORT,Macd_Updown_Signals,D
225,^DJI,2021-02-26,HOLD SHORT,Macd_Updown_Signals,D
226,^GSPC,2021-02-26,HOLD SHORT,Macd_Updown_Signals,D


## 周线 - MACD

In [9]:
all_csvs = "C:/Users/Administrator/CE_github_2/data_pipeline/Data/*.csv"
stock_list = []
date_list = []
signal_list = []
len_ = 1
date_col = "Date"
price_col = "Adj Close"
signal_df = pd.DataFrame()
model_name = "Macd_Updown_Signals"
model_freq = "W"
total_number = len(glob.glob(all_csvs)[:])


for fname in glob.glob(all_csvs)[:]:
    print ("Calculating Signals for No.{} : {}".format(len_, fname))
    data = pd.read_csv(fname)
    data.index = pd.to_datetime(data[date_col])
    data = data.resample(model_freq).last()
    data.reset_index(drop = True, inplace = True)
    try:
        data_macd = MACD(data, "Adj Close", 12, 26)
        ticker, last_date, last_signal = macd_updown_signals(data_macd, "MACD", "Date", 'Ticker')
        stock_list.append(ticker)
        date_list.append(last_date)
        signal_list.append(last_signal)
    except Exception as e:
        print(e)
    len_+= 1
signal_df['Ticker'] = stock_list
signal_df['Last_Date'] = date_list
signal_df['Signal'] = signal_list
signal_df['model_name'] = model_name
signal_df['model_freq'] = model_freq
signal_df_weekly = signal_df.copy()
print("All Done!")

Calculating Signals for No.1 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\000001.SS.csv
Calculating Signals for No.2 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0005.HK.csv
Calculating Signals for No.3 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0008.HK.csv
Calculating Signals for No.4 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0142.HK.csv
Calculating Signals for No.5 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0151.HK.csv
Calculating Signals for No.6 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0175.HK.csv
Calculating Signals for No.7 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0186.HK.csv
Calculating Signals for No.8 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0215.HK.csv
Calculating Signals for No.9 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0241.HK.csv
Calculating Signals for No.10 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\0268.HK.csv
Calculating Signals for No.

Calculating Signals for No.90 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\ADBE.csv
Calculating Signals for No.91 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\ADP.csv
Calculating Signals for No.92 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\ADSK.csv
Calculating Signals for No.93 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\AEMD.csv
Calculating Signals for No.94 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\AHPI.csv
Calculating Signals for No.95 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\ALB.csv
Calculating Signals for No.96 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\AMD.csv
Calculating Signals for No.97 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\AMZN.csv
Calculating Signals for No.98 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\APT.csv
Calculating Signals for No.99 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\ARKF.csv
Calculating Signals for No.100 : C:/Users/Administrato

Calculating Signals for No.184 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\RNG.csv
Calculating Signals for No.185 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\RUN.csv
Calculating Signals for No.186 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SAP.csv
Calculating Signals for No.187 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SBE.csv
Calculating Signals for No.188 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SHOP.csv
Calculating Signals for No.189 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SMAR.csv
Calculating Signals for No.190 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SPLK.csv
Calculating Signals for No.191 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SPY.csv
Calculating Signals for No.192 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SQ.csv
Calculating Signals for No.193 : C:/Users/Administrator/CE_github_2/data_pipeline/Data\SQM.csv
Calculating Signals for No.194 : C:/Users/Admini

In [10]:
merged_daily_weekly = signal_df_daily.merge(signal_df_weekly, on = "Ticker", how = "outer")
daily_weekly_buy = merged_daily_weekly[((merged_daily_weekly['Signal_x']=="BUY")&(merged_daily_weekly['Signal_y']=='BUY')) |\
                                       ((merged_daily_weekly['Signal_x']=='BUY')&(merged_daily_weekly['Signal_y']=='HOLD LONG'))]

daily_weekly_short = merged_daily_weekly[((merged_daily_weekly['Signal_x']=="SELL")&(merged_daily_weekly['Signal_y']=='SELL')) |\
                                       ((merged_daily_weekly['Signal_x']=='SELL')&(merged_daily_weekly['Signal_y']=='HOLD SHORT'))]

In [11]:
signal_path = "C:/Users/Administrator/CE_github_2/data_pipeline/Signals/"
signal_file = signal_path+model_name+"_"+last_date+"_D_W"+".csv"
merged_daily_weekly.to_csv(signal_file)

In [12]:
daily_weekly_buy

,Ticker,Last_Date_x,Signal_x,model_name_x,model_freq_x,Last_Date_y,Signal_y,model_name_y,model_freq_y
76,6869.HK,2021-02-26,BUY,Macd_Updown_Signals,D,2021-02-26,HOLD LONG,Macd_Updown_Signals,W


In [13]:
daily_weekly_short

,Ticker,Last_Date_x,Signal_x,model_name_x,model_freq_x,Last_Date_y,Signal_y,model_name_y,model_freq_y
18,0522.HK,2021-02-26,SELL,Macd_Updown_Signals,D,2021-02-26,HOLD SHORT,Macd_Updown_Signals,W
33,0981.HK,2021-02-26,SELL,Macd_Updown_Signals,D,2021-02-26,SELL,Macd_Updown_Signals,W
36,1137.HK,2021-02-26,SELL,Macd_Updown_Signals,D,2021-02-26,HOLD SHORT,Macd_Updown_Signals,W
77,6969.HK,2021-02-26,SELL,Macd_Updown_Signals,D,2021-02-26,HOLD SHORT,Macd_Updown_Signals,W
116,CSCO,2021-02-26,SELL,Macd_Updown_Signals,D,2021-02-26,HOLD SHORT,Macd_Updown_Signals,W
132,FXE,2021-02-26,SELL,Macd_Updown_Signals,D,2021-02-26,HOLD SHORT,Macd_Updown_Signals,W
149,JNJ,2021-02-26,SELL,Macd_Updown_Signals,D,2021-02-26,HOLD SHORT,Macd_Updown_Signals,W
162,NFLX,2021-02-26,SELL,Macd_Updown_Signals,D,2021-02-26,HOLD SHORT,Macd_Updown_Signals,W
174,PFE,2021-02-26,SELL,Macd_Updown_Signals,D,2021-02-26,HOLD SHORT,Macd_Updown_Signals,W


# 2. BOLL布林线系列

In [ ]:
def BBANDS(df, price_col, num_of_std): # n = 20
    """
    Bollinger Bands, rationale CHECKED, code CHECKED
    Create the common bbands which I understand instead of using the one from Quantopian forum.
    """
    rlng_mean = df[price_col].ewm(span = 20).mean().rename('Rolling_{}_Day_Mean'.format(20))
    rlng_std  = df[price_col].rolling(window = 20).std()
    u_band = (rlng_mean + rlng_std*num_of_std)
    l_band = (rlng_mean - rlng_std*num_of_std)
    df['MA_20'] = rlng_mean
    df['LOW_BAND'] = l_band
    df['UP_BAND'] = u_band
    return df

In [ ]:
def bolling_upcross_lband(df, price_col, date_col, ticker_col):
    ticker = df[ticker_col].values[-1]
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    del df['index']
    listLongShort = []
    listDate = []
    close = df[price_col].values
    up_band = df['UP_BAND'].values
    low_band = df['LOW_BAND'].values
    
    for i in range(1, len(df)):
        last_date = df[date_col][i]
        
        if close[i]>low_band[i] and close[i-1]<low_band[i-1] and close[i-2]>low_band[i-2]:
            listLongShort.append("BUY")
            listDate.append(last_date)
        else:
            listLongShort.append("NO SIGNAL")
            listDate.append(last_date)
            
    return ticker, listDate[-1], listLongShort[-1]

In [ ]:
all_csvs = "C:/Users/Administrator/CE_github_2/data_pipeline/Data/*.csv"
stock_list = []
date_list = []
signal_list = []
len_ = 1
date_col = "Date"
price_col = "Adj Close"
signal_df = pd.DataFrame()
model_name = "Boll_Upcross_Lband"
total_number = len(glob.glob(all_csvs)[:])


for fname in glob.glob(all_csvs)[:]:
    print ("Calculating Signals for No.{} : {}".format(len_, fname))
    data = pd.read_csv(fname)
    try:
        data_boll = BBANDS(data, "Adj Close", 2)
        ticker, last_date, last_signal = bolling_upcross_lband(data_boll, "Adj Close", "Date", 'Ticker')
        stock_list.append(ticker)
        date_list.append(last_date)
        signal_list.append(last_signal)
    except Exception as e:
        print(e)
    len_+= 1
signal_df['Ticker'] = stock_list
signal_df['Last_Date'] = date_list
signal_df['Signal'] = signal_list
signal_df['model_name'] = model_name
print("All Done!")
# def merge_df(df1, df2):
#     df1.sort_values(date_col, inplace = True)
#     merged = df1.merge(df2, on = date_col, how = 'outer')
#     merged.sort_values(date_col, inplace = True)
#     return merged

# merged_all = reduce(merge_df, stock_list)
# merged_all.set_index(date_col, inplace=True)

In [ ]:
signal_path = "C:/Users/Administrator/CE_github_2/data_pipeline/Signals/"
signal_file = signal_path+model_name+"_"+last_date+".csv"
signal_df.to_csv(signal_file)

# 跨周期信号代码：

In [ ]:
# def MACD(df, n_fast, n_slow, n_macd, price_col): # n_fast = 12, n_slow = 26
#     """
#     http://stockcharts.com/docs/doku.php?id=scans:indicators
#     MACD, MACD Signal and MACD difference, rationale CHECKED, code CHECKED, updated
#     # Conventional look-back window for calculating MACDsign is 9
#     """
#     EMAfast = df[price_col].ewm(span = n_fast, min_periods = n_fast - 1).mean()
#     EMAslow = df[price_col].ewm(span = n_slow, min_periods = n_slow - 1).mean()
#     MACD = pd.Series(EMAfast - EMAslow, name = 'MACD_' + str(n_fast) + '_' + str(n_slow))
#     MACDsign = MACD.ewm(span = n_macd, min_periods = n_macd-1).mean().rename('MACDsign_' + str(n_fast) + '_' + str(n_slow))
#     MACDdiff = pd.Series(MACD - MACDsign, name = 'MACDdiff_' + str(n_fast) + '_' + str(n_slow))
#     df['MACD_Diff'] = MACD
#     df['MACD_Diff_EMA'] = MACDsign
#     df['MACD'] = MACDdiff
#     df['SIGNAL_STATUS'] = df['MACD'].apply(lambda x: "多头状态" if x>0 else ("空头状态" if x<0 else "无信号状态"))
#     return df

In [ ]:
# indicator_df = MACD(df, 12, 26, 9, 'close')
# indicator_df.index = pd.to_datetime(indicator_df['trade_date'])

In [ ]:
# # 查看周线：

# period_type = "W"
# df_copy = df.copy()
# weekly_df = df_copy.resample(period_type).last()
# weekly_indicator_df = MACD(weekly_df, 12, 26, 9, 'close')

# last_daily_signal = indicator_df['SIGNAL_STATUS'].values[-1]
# last_weekly_signal = weekly_indicator_df['SIGNAL_STATUS'].values[-1]
# last_price_vol_date = indicator_df['trade_date'].values[-1]